Notebook: Exploration of csv files

In [44]:
"""Exploration notebook for data analysis.

This notebook contains data exploration steps for disaster analysis.
"""

import sys
from pathlib import Path

import pandas as pd

from src.data_consolidation.dictionary import STANDARD_COLUMNS

module_path = Path("..").resolve()
sys.path.append(str(module_path))

In [46]:
def read_dat(dat_file: str) -> pd:
    """Reads a CSV file from the data_prep directory."""
    dat_dir = Path("../data_prep/").resolve()
    dat_path = dat_dir / dat_file
    return pd.read_csv(dat_path)

In [47]:
glide_prep_df = read_dat("glide_prep.csv")
gdacs_prep_df = read_dat("gdacs_prep.csv")
emdat_prep_df = read_dat("emdat_prep.csv")
disaster_charter_df = read_dat("disaster_charter_prep.csv")
cerf_df = read_dat("cerf_prep.csv")
idmc_df = read_dat("idmc_prep.csv")
ifrc_df = read_dat("ifrc_prep.csv")

In [48]:
pre_dfs = [
    glide_prep_df,
    gdacs_prep_df,
    emdat_prep_df,
    disaster_charter_df,
    cerf_df,
    idmc_df,
    ifrc_df,
]

In [49]:
for i, df in enumerate(pre_dfs):
    missing_cols = set(STANDARD_COLUMNS) - set(df.columns)
    for col in missing_cols:
        df[col] = None
    df_standard = df[STANDARD_COLUMNS]
    pre_dfs[i] = df_standard

all_data = pd.concat(pre_dfs, ignore_index=True)
all_data["Date"] = pd.to_datetime(all_data["Date"], errors="coerce")
group_key = ["Event_Type", "Country"]

/tmp/ipykernel_18473/1884474460.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_data = pd.concat(pre_dfs, ignore_index=True)
